In [52]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
	using Flux: @functor
	using ..Transformers: Abstract3DTensor, Container, epsilon, batchedmul, batched_triu!
	using NNlib
	using Tullio, ForwardDiff, Zygote
	using Einsum
end

In [53]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end


Vocabulary{String}(13, unk=0)

In [54]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [55]:
#nichts ändern, generieren 10 string token, nutze 1-10

sample_data()

(["2", "8", "8", "3", "10", "4", "7", "9", "5", "6"], ["2", "8", "8", "3", "10", "4", "7", "9", "5", "6"])

In [56]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [57]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data, +3
    end

sample_ex = preprocess.(sample_data()) = (["11", "10", "1", "1", "2", "6", "9", "1", "5", "4", "9", "12"], ["11", "10", "1", "1", "2", "6", "9", "1", "5", "4", "9", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 13, 4, 4, 5, 9, 12, 4, 8, 7, 12, 3]


12-element Vector{Int64}:
  2
 13
  4
  4
  5
  9
 12
  4
  8
  7
 12
  3

In [58]:
#nichts ändern
sample = preprocess.(sample_data())


(["11", "2", "10", "7", "7", "5", "5", "6", "9", "10", "8", "12"], ["11", "2", "10", "7", "7", "5", "5", "6", "9", "10", "8", "12"])

In [59]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
  5
 13
 10
 10
  8
  8
  9
 12
 13
 11
  3

In [60]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

Embed(512)

In [61]:
#define a position embedding layer mentioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu


PositionEmbedding(512)

In [62]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [63]:
abstract type AbstractAttention end

In [64]:
create_atten_mask1(T::Type, score::AbstractArray, ::Nothing, future) = create_atten_mask1(T, score, fill!(similar(score, size(score,1), size(score, 2), 1), one(T)), future)
function create_atten_mask1(T::Type, score::AbstractArray, _mask::AbstractArray, future::Bool=false)
  #size(mask) == (q, k, n, b)

  # ql, kl = size(mask)
  mask = copy(_mask)

  maskval = convert(T, -1e9)
  !future && batched_triu!(mask, 0)
  mask .= (1 .- mask) .* maskval
  return mask
end

create_atten_mask1 (generic function with 3 methods)

In [65]:
Flux.@nograd create_atten_mask1

In [66]:
struct MultiheadAttention_1{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end

In [67]:
Flux.functor(mh::MultiheadAttention_1) = (mh.iqproj, mh.ikproj, mh.ivproj, mh.oproj), m -> MultiheadAttention_1(mh.head, mh.future, m..., mh.drop)

In [68]:
MultiheadAttention_1(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = MultiheadAttention_1(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, mh::MultiheadAttention_1)
    hs = div(size(mh.iqproj.weight)[1], mh.head)
    is = size(mh.iqproj.weight)[end]
    os = size(mh.oproj.weight)[1]

    print(io, "MultiheadAttention(")
    print(io, "head=$(mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(mh.drop.p))")
    else
        print(io, ")")
    end
end



In [69]:
function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}
qs = size(query)
ks = size(key)
vs = size(value)

#size(ipq) == (h, q_seq_len, batch)
ipq = @toNd mh.iqproj(query)
ipk = @toNd mh.ikproj(key)
ipv = @toNd mh.ivproj(value)

h = size(ipq, 1)
hs = div(h, mh.head)

#size(ipq) == (hs, q_seq_len, head, batch)
ipq = permutedims(reshape(ipq, hs, mh.head, qs[2], qs[3]), [1, 3, 2, 4])
ipk = permutedims(reshape(ipk, hs, mh.head, ks[2], ks[3]), [1, 3, 2, 4])
ipv = permutedims(reshape(ipv, hs, mh.head, vs[2], vs[3]), [1, 3, 2, 4])

#size(ipq) == (hs, q_seq_len, head * batch)
ipq = reshape(ipq, hs, qs[2], :)
ipk = reshape(ipk, hs, ks[2], :)
ipv = reshape(ipv, hs, vs[2], :)

atten = attention1(ipq,ipk,ipv,
mask,
mh.future,
mh.drop)

atten = permutedims(reshape(atten, hs, qs[2], mh.head, qs[3]), [1, 3, 2, 4]) #size(atten) == (hs, head, ql, b)
atten = reshape(atten, h, qs[2], qs[3]) #size(atten) == (h, ql, b)

out = @toNd mh.oproj(atten)
out #size(out) == (h, q_seq_len, batch)
end

function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: AbstractMatrix{T},
                         A2 <: AbstractMatrix{T},
                         A3 <: AbstractMatrix{T}}

# size(query) == (dims, seq_len)
ipq = mh.iqproj(query)
ipk = mh.ikproj(key)
ipv = mh.ivproj(value)

h = size(ipq)[1] #h == hs * head
hs = div(h, mh.head)

#size(hq) == (hs, seq_len, head)
hq = permutedims(reshape(ipq, hs, mh.head, :), [1, 3, 2])
hk = permutedims(reshape(ipk, hs, mh.head, :), [1, 3, 2])
hv = permutedims(reshape(ipv, hs, mh.head, :), [1, 3, 2])

atten = attention1(hq, hk, hv,
mask,
mh.future,
mh.drop)

# size(atten) == (head*hs, seq_len)
atten = reshape(permutedims(atten, [1, 3, 2]), h, :)

mh.oproj(atten)
end

In [70]:
function apply_mask1(score, mask)
    s = size(score)
    ms = size(mask)
    bxn = s[end]
    b = ms[end]
    if bxn == b || b == 1
      return score .+ mask
    else
      return reshape(reshape(score, s[1:end-1]..., :, b) .+
                     reshape(mask, ms[1:end-1]..., 1, b), s)
    end
  end
  
  apply_mask1(score::AbstractArray{T}, ::Nothing, future) where T = future ? score : apply_mask1(score, create_atten_mask1(T, score, nothing, future))
  apply_mask1(score::AbstractArray{T}, mask, future) where T = apply_mask(score, create_atten_mask1(T, score, mask, future))

apply_mask1 (generic function with 3 methods)

In [71]:
# elu feature map used by Katharopoulos et al.
function oftf(x, y)
    oftype(float(x), y)
end
function nelu(x, α=1)
    ifelse(x ≥ 0, float(x)+1, @fastmath oftf(x, α) * (exp(x) - 1)+1)
end

nelu (generic function with 2 methods)

In [72]:
x = embedding(encoded_sample)

512×12 Matrix{Float32}:
  0.532366    -0.340092   -1.00637      …  -0.0046613   0.842006
  0.78598      0.943564    0.28563         -0.960923   -0.872406
  0.53659     -0.356066   -0.937938        -0.430995    0.574782
  0.812761     1.05137     0.372316        -0.732524   -0.969798
  0.513482    -0.2457     -0.971396        -0.655359    0.247136
  0.776838     0.964129    0.475126     …  -0.450193   -0.929151
  0.649673    -0.233105   -0.872058        -0.93654    -0.190447
  0.757951     0.983154    0.53187         -0.13585    -0.762554
  0.721112    -0.192795   -0.869488        -0.997535   -0.555453
  0.720656     1.02114     0.511187         0.253685   -0.564138
  ⋮                                     ⋱   ⋮          
 -0.00464669   0.0816127   0.0661733        0.0272452  -0.0288252
  0.956741     1.0345      0.953582         0.983212    1.06201
  0.0218737   -0.0443764  -0.000606865  …  -0.0153725  -0.00262515
  1.00652      1.01619     1.05776          1.04069     0.972271
  0.0307

In [73]:
function splitHeads(x, batch_size, head, depth)
    x = reshape(x, (batch_size, :, head, depth))
end

splitHeads (generic function with 1 method)

In [74]:
function padding_mask(seq)
    # Returns (batch, 1, seq_len, 1) tensor with 1's where the sequence is padded, 0 where it is not
    mask = isequal.(seq, 0)
    mask = mask[:,1,:,1]
    na = [CartesianIndex()]
return Float32.(mask[:,na,:,na])  # (batch, 1, seq_len, 1) m l j h  <- j gets masked
end

padding_mask (generic function with 1 method)

In [75]:
#hier anstatt softmax linear
function attention1(query::A1,
    key::A2,
    value::A3,
    mask, future::Bool,
    dropout) where {T,
                    A1 <: Abstract3DTensor{T},
                    A2 <: Abstract3DTensor{T},
                    A3 <: Abstract3DTensor{T}}
#size(query) == (dims, {q,k}_seq_len, batch) == size(key) == size(value)
#size(ipq) == (hs, q_seq_len, head * batch)
#size(score) == (k_seq_len, q_seq_len, batch)
#=
dk = size(key, 1)
score = batchedmul(key, query; transA = true) 
score = score ./ convert(T, sqrt(dk))

score = apply_mask1(score, mask, future)
score = softmax(score; dims=1)
dropout !== nothing && (score = dropout(score))
batchedmul(value, score) #size(return) == (dims, q_seq_len, batch)
=#
#size(query) == (batch, {q,k}_seq_len, dims) == size(key) == size(value)
query = permutedims(query,(3,2,1))
key = permutedims(key,(3,2,1))
value = permutedims(value,(3,2,1))

qs = size(query)
ks = size(key)
vs = size(value)

#depth = div(mh.size, mh.head)
depth = div(128, 8)   

#query = nelu(splitHeads(query, qs[1], mh.head, depth))
#key = nelu(splitHeads(key, qs[1], mh.head, depth))
#value = splitHeads(value, qs[1], mh.head, depth)

# size(key) == (batch_size,  seq_len_k, num_heads, depth_q) (m,j,h,d)
query = nelu.(splitHeads(query, qs[1], 8, depth))
key = nelu.(splitHeads(key, ks[1], 8, depth))
value = splitHeads(value, vs[1], 8, depth)

# size(k_v) == (batch_size, depth_k, depth_v, seq_len_v)
@tullio k_v[m,d,e,h] := key[m,j,h,d]*value[m,j,h,e]

padmask = padding_mask(key)

key = key .* padmask

k_reduced = dropdims(sum(key, dims=2), dims=2)

k_reduced = k_reduced .+ 1e-8

@tullio z_1[m,l,h] := query[m,l,h,d]*k_reduced[m,h,d]
# size(z) == (batch_size, num_heads, seq_len_q)
z = 1 ./ z_1 # ...

# size(output) == (batch_size,len_q, heads, depth_v)
@tullio output[m,l,h,e] := query[m,l,h,d]*k_v[m,d,e,h]*z[m,l,h]

#output = reshape(output, (qs[1], qs[2], mh.head*depth))
# size(output) == (batch_size,len_q, d_model)
output = reshape(output, (qs[1], :, 8*depth))

output = permutedims(output,(3,2,1))
#(dims, q_seq_len, batch)
end


attention1 (generic function with 1 method)

In [76]:
abstract type AbstractTransformer end

In [77]:
struct PwFFN{Di<:Dense, Do<:Dense}
    din::Di
    dout::Do
end

@functor PwFFN


"just a wrapper for two dense layer."
function PwFFN(size::Int, h::Int, act = nelu)
    PwFFN(
    Dense(size, h, act),
    Dense(h, size)
)
end
function (pw::PwFFN)(x::AbstractMatrix)
  #size(x) == (dims, seq_len)
  pw.dout(pw.din(x))
end

function (pw::PwFFN)(x::A) where {T, N, A<:AbstractArray{T, N}}
  new_x = reshape(x, size(x, 1), :)
  y = pw(new_x)
  return reshape(y, Base.setindex(size(x), size(y, 1), 1))
end

In [78]:
struct Transformer1{MA<:MultiheadAttention_1, LA<:LayerNorm, P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
    mh::MA
    mhn::LA
    pw::P
    pwn::LP
    drop::DP
end

@functor Transformer1


"""
    Transformer(size::Int, head::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)
    Transformer(size::Int, head::Int, hs::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)  

Transformer layer.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
When `future` is `false`, the k-th token can't see the j-th tokens where j > k. `pdrop` is the dropout rate.
"""

########
#ganze Funktion von Transformer ändern
########
function Transformer1(size::Int, head::Int, ps::Int; future::Bool = true, act = relu, pdrop = 0.1)  #relu vielleicht wieder auf elu ändern
    rem(size, head) != 0 && error("size not divisible by head")
    Transformer1(size, head, div(size, head), ps;future=future, act=act, pdrop=pdrop)
end

Transformer1(size::Int, head::Int, hs::Int, ps::Int; future::Bool = true, act = relu, pdrop = 0.1) = Transformer1(
    MultiheadAttention_1(head, size, hs, size; future=future, pdrop=pdrop),
    LayerNorm(size),
    PwFFN(size, ps, act),
    LayerNorm(size),
    Dropout(pdrop),     #braucht man das? vielleicht nicht
)

function (t::Transformer1)(x::A, mask=nothing) where {T, N, A<:AbstractArray{T, N}}
    dropout = t.drop
    a = t.mh(x, x, x; mask=mask)
    a = dropout(a)
    res_a = x + a
    res_a = t.mhn(res_a)
    pwffn = t.pw(res_a)
    pwffn = dropout(pwffn)
    res_pwffn = res_a + pwffn
    res_pwffn = t.pwn(res_pwffn)
    res_pwffn
end

function Base.show(io::IO, t::Transformer1) 
    hs = div(size(t.mh.iqproj.weight)[1], t.mh.head)
    h, ps = size(t.pw.dout.weight)

    print(io, "Transformer(")
    print(io, "head=$(t.mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "pwffn_size=$(ps), ")
    print(io, "size=$(h)")
    if Flux.istraining()
        print(io, ", dropout=$(t.drop.p))")
    else
        print(io, ")")
    end
end

In [79]:
###########
    #auch diese Funktion ändern
    ###############
    struct TransformerDecoder1{MA<:MultiheadAttention_1, LA<:LayerNorm,
        IMA<:MultiheadAttention_1, ILA<:LayerNorm,
        P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
mh::MA
mhn::LA
imh::IMA
imhn::ILA
pw::P
pwn::LP
drop::DP
end

@functor TransformerDecoder1

"""
TransformerDecoder(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
TransformerDecoder(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1)

TransformerDecoder layer. Decode the value from a Encoder.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
`pdrop` is the dropout rate.
"""
function TransformerDecoder1(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
rem(size, head) != 0 && error("size not divisible by head")
TransformerDecoder1(size, head, div(size, head), ps; act=act, pdrop=pdrop)
end

TransformerDecoder1(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1) = TransformerDecoder1(
MultiheadAttention_1(head, size, hs, size; future=false, pdrop=pdrop),
LayerNorm(size),
MultiheadAttention_1(head, size, hs, size; future=true, pdrop=pdrop), #future = true --> Unterschied zu oben??
LayerNorm(size),
PwFFN(size, ps, act),
LayerNorm(size),
Dropout(pdrop),
)

function (td::TransformerDecoder1)(x::AbstractArray{T,N}, m, mask=nothing) where {T,N}
dropout = td.drop
a = td.mh(x,x,x)
a = dropout(a)
res_a = x + a
res_a = td.mhn(res_a)

ia = td.imh(res_a, m, m, mask=mask)
ia = dropout(ia)
res_ia = res_a + ia
res_ia = td.imhn(res_ia)

pwffn = td.pw(res_ia)
pwffn = dropout(pwffn)
res_pwffn = res_ia + pwffn
res_pwffn = td.pwn(res_pwffn)
res_pwffn
end

function Base.show(io::IO, td::TransformerDecoder1)
hs = div(size(td.imh.iqproj.weight)[1], td.imh.head)
h, ps = size(td.pw.dout.weight)

print(io, "TransformerDecoder(")
print(io, "head=$(td.mh.head), ")
print(io, "head_size=$(hs), ")
print(io, "pwffn_size=$(ps), ")
print(io, "size=$(h)")
if Flux.istraining()
print(io, ", dropout=$(td.drop.p))")
else
print(io, ")")
end
end

In [80]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [81]:
encode_t2 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [82]:
encode_t3 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [83]:
encode_t4 = Transformer1(512, 8, 64, 2048) |> gpu


Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [84]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [85]:
decode_t2 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [86]:
decode_t3 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [87]:
decode_t4 = TransformerDecoder1(512, 8, 64, 2048, act=nelu) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [88]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [89]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    t2 = encode_t2(t1)
    t3 = encode_t2(t2)
    t4 = encode_t2(t3)
    return t2
  end

encoder_forward (generic function with 1 method)

In [90]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    t2 = decode_t2(t1, m)
    t3 = decode_t3(t2, m)
    t4 = decode_t4(t3, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [91]:
#bleibt so
enc = encoder_forward(encoded_sample);

In [92]:
#bleibt so
probs = decoder_forward(encoded_sample, enc);

In [93]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [94]:
Flux.@nograd smooth

In [95]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    #l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])
    l = logcrossentropy(label[:, 2:end, :], probs[:, 1:end-1, :])  
    # logcrossentropy used by Katharopoulos et al.
    return l
  end

loss (generic function with 1 method)

In [96]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, encode_t2, encode_t3, encode_t4, decode_t1, decode_t2, decode_t3, decode_t4, linear);

In [97]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), IdDict{Any, Any}())

In [98]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:1000
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [99]:
train!()


loss = 78.7365860298736
loss = 76.88064219789167
loss = 74.04303201652364
loss = 74.11155688418785
loss = 73.57216960806478
loss = 73.36253420773416
loss = 72.72487034497733
loss = 72.48083316099495
loss = 72.09910250880779
loss = 71.11688390444527
loss = 67.93142804623506
loss = 67.91772373418061
loss = 67.63002707883186
loss = 67.63064346683228
loss = 67.49489492723916
loss = 67.70096465888821
loss = 67.07917178463154
loss = 67.49601940100995
loss = 67.79665869255831
loss = 67.90064429938103
loss = 67.95881488686763
loss = 67.9300916848299
loss = 67.23356441766379
loss = 67.62499188601504
loss = 67.17462278671292
loss = 68.16663054219269
loss = 67.5596433946674
loss = 67.59442323879216
loss = 68.36360446391588
loss = 67.55308990915736
loss = 66.88707013034373
loss = 68.2677565086086
loss = 68.2658049526346
loss = 66.44084817844094
loss = 67.76113502707248
loss = 67.43725365983774
loss = 67.29189681756512
loss = 67.25200907153643
loss = 67.40337531880574
loss = 67.84304762951321
loss 

┌ Info: start training
└ @ Main c:\Users\tyto4\OneDrive\Dokumente\GitHub\test\LinAt.ipynb:5


In [100]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

translate (generic function with 1 method)

In [101]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

DimensionMismatch: DimensionMismatch("array size 5632 must be divisible by the product of the new dimensions (8, Colon(), 8, 16)")